The Task is to extract relevant infromation from the messy medical data using regex.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

1) Extract the date strings
Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

2) Normalize the extracted dates considering the following rules:  
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

3) Sort records in ascending chronological order accoring.

In [127]:
from datetime import datetime
import re
import pandas as pd

In [139]:
def extract_dates_from_notes(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()

    date_strings = []
    unmatched_lines = []

    for i, line in enumerate(data, start=1):
        # Use regex to find date patterns in the line
        matches = re.findall(r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|'    #dd-mm-yyyy, dd/mm/yyyy, d-m-yy
                             r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.\s]\d{1,2}(?:th|st|nd|rd)?[-,\s]\d{2,4}|' #Month dd yyyy, Month-dd-yyyy
                             r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2,4})?|'     # dd Month yyyy, dd Month yyyy
                             r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2},?) (?:\d{2,4})?|'  #dd/yyyy
                             r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}th|st|nd|rd),? (?:\d{2,4})?|' #dd/mm/yyyy, d/m/yy
                             r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,? (?:\d{2,4})?|'
                             r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}nd|st|rd),? (?:\d{2,4})?|'
                             r'\d{1,2}/\d{4}|'
                             r'\d{1,2}/\d{1,2}/\d{2,4}|'
                             r'\d{4})\b', line)
        
        # If no matches are found, add the line to unmatched_lines
        if not matches:
            unmatched_lines.append((i, line))
        
        date_strings.extend((match, i) for match in matches)

    if unmatched_lines:
        print("Unmatched Lines:")
        for line_number, line_content in unmatched_lines:
            print(f"{line_number}: {line_content}")

    return date_strings

In [136]:
def process_and_normalize_dates(file_path, output_path):
    date_strings = extract_dates_from_notes(file_path)

    # Normalize dates and filter out None values
    normalized_dates = [(normalize_date(date_str), line) for date_str, line in date_strings if normalize_date(date_str) is not None]

    df = pd.DataFrame(normalized_dates, columns=["res", "index"])
    df.to_csv(output_path, index=False)

    return normalized_dates

In [140]:
file_path = 'E:\\University\\Інше\\IV Amazinum Data Science Camp\\Lecture 21\\dates.txt'
output_path = 'E:\\University\\Інше\\IV Amazinum Data Science Camp\\Lecture 21\\normalized_dates.csv'

normalized_dates = process_and_normalize_dates(file_path, output_path)
df = pd.read_csv(output_path)
df

,res,index
0,1993-03-25,1
1,1985-06-18,2
2,1971-07-08,3
3,1975-09-27,4
4,1996-02-06,5
...,...,...
397,1979-01-01,496
398,2006-01-01,497
399,2008-01-01,498
400,2005-01-01,499
